In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')   # connect to gdrive

# Change working directory to be current folder
import os
os.chdir('/content/gdrive/My Drive/Practice/4/ml-arxiv-papers')


Mounted at /content/gdrive


 # Evaluate the model performance


- semantic similarity (METEOR and BERTScore)
- word and phrase level overlap (BLEU and ROUGE scores)
- human evaluation

In [ ]:
!pip install rouge
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
# semantic similarity (METEOR and BERTScore)
# word and phrase level overlap (BLEU and ROUGE scores)

import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from bert_score import score

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')

# Load JSON file
with open("./test_dataset_results.json", "r") as f:
    results = json.load(f)

# Initialize variables to store total scores
total_bert_score = 0
total_meteor_score = 0
total_bleu_score = 0
total_rouge_score = {'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0.0}

# Loop through each result and calculate scores
for result in results:
    answer = result['answer']
    ground_truth = result['ground_truth']

    # Tokenize hypothesis and reference
    hypothesis_tokens = word_tokenize(answer)
    reference_tokens = word_tokenize(ground_truth)

    # BERTScore
    P, R, F1 = score([answer], [ground_truth], lang='en', verbose=False)
    total_bert_score += F1.item()

    # METEOR
    meteor = meteor_score([reference_tokens], hypothesis_tokens)
    total_meteor_score += meteor

    # BLEU
    bleu_score = sentence_bleu([reference_tokens], hypothesis_tokens)
    total_bleu_score += bleu_score

    # ROUGE
    rouge = Rouge()
    rouge_scores = rouge.get_scores(answer, ground_truth)[0]
    for metric, scores in rouge_scores.items():
        total_rouge_score[metric] += scores['f']

# Calculate average scores
num_results = len(results)
average_bert_score = total_bert_score / num_results
average_meteor_score = total_meteor_score / num_results
average_bleu_score = total_bleu_score / num_results
average_rouge_score = {metric: score / num_results for metric, score in total_rouge_score.items()}

# Print average scores
print("Average BERTScore:", average_bert_score)
print("Average METEOR score:", average_meteor_score)
print("Average BLEU score:", average_bleu_score)
print("Average Rouge score:", average_rouge_score)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights 

Average BERTScore: 0.8681818187236786
Average METEOR score: 0.381453353813237
Average BLEU score: 0.1394442421655174
Average Rouge score: {'rouge-1': 0.3415367757422849, 'rouge-2': 0.19690651144858345, 'rouge-l': 0.33016678744845995}


分析和评估这个结果

ChatGPT

这些结果是对模型性能的评估，以下是针对每个指标的分析和评估：

- Average BERTScore (0.8682):

BERTScore是用于衡量生成文本和参考文本之间语义相似性的指标，它利用预训练的BERT模型对句子进行编码，并计算它们之间的相似性分数。这里的平均BERTScore为0.8682，表明模型生成的文本与参考文本之间的语义相似性较高。


- Average METEOR Score (0.3815):

METEOR分数是另一种评估机器翻译质量的指标，它考虑了单词级别的对齐以及句子级别的语义相似性。平均METEOR分数为0.3815，相对较高，表明模型生成的文本在一定程度上与参考文本相符合。

- Average BLEU Score (0.1394):

BLEU分数用于评估机器翻译的质量，其范围通常在0到1之间，其中1表示完美匹配。这里的平均BLEU分数约为0.1394，这意味着模型生成的文本与参考文本之间的匹配程度相对较低。可能的原因包括词汇选择、句法结构等方面的差异。


- Average Rouge Score:

ROUGE分数用于评估生成文本和参考文本之间的重叠程度，包括单词级别和句子级别的重叠。这里提供了三个ROUGE指标的平均分数：

rouge-1: 平均值约为0.3415，表示模型生成的单个词语与参考文本中的单个词语之间的重叠较好。

rouge-2: 平均值约为0.1969，表示模型生成的两个词语组成的短语与参考文本中的短语之间的重叠较低。

rouge-l: 平均值约为0.3302，表示模型生成的文本与参考文本之间的最长公共子序列的长度较高，即在句子级别上的重叠较好。

综合来看，模型在语义相似性方面表现良好（METEOR和BERTScore较高），但在词语和短语级别的重叠方面可能存在改进空间（BLEU和ROUGE分数相对较低）。可能的改进方法包括模型调优、更好的训练数据、改进的生成策略等。